# Comparador de coches

Mostrar en gráficos de radar las características más importantes para poder comparar 2 coches. Usa una función de escalado de datos para poder hacer la comparación entre diferentes escalas. 

**Extra: Añadir la primera foto del coche.**



In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import base64

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [125]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded = base64.b64encode(image_file.read()).decode()
    return f"data:image/png;base64,{encoded}"


In [126]:
df = pd.read_csv("../data/coches_segunda_mano-19-02-2025_limpio.csv")

In [127]:
df_comparar: pd.DataFrame = df[["modelo_titulo","consumo_medio","precio", "potencia", "kilometraje","ruta_imagen"]]

print(len(df_comparar))

coche_a = df_comparar.iloc[2]
coche_a = coche_a.fillna(0)

coche_b = df_comparar.iloc[33]
coche_b =coche_b.fillna(0)


4778


In [128]:
'''quitamos las variables modelo_titulo y ruta_imagen ya que no son utiles para comparar las características'''
categorias = coche_a[['consumo_medio', 'precio', 'potencia', 'kilometraje']].keys()
valores_a = coche_a[categorias].values
valores_b = coche_b[categorias].values
imagen_a = encode_image(coche_a['ruta_imagen'])
imagen_b = encode_image(coche_b['ruta_imagen'])

scaler = StandardScaler()
valores_a: np.ndarray = scaler.fit_transform(valores_a.reshape(-1,1))
valores_b: np.ndarray = scaler.fit_transform(valores_b.reshape(-1,1))

df_comparar = pd.DataFrame({coche_a['modelo_titulo'] :valores_a.flatten(),
                            coche_b['modelo_titulo'] :valores_b.flatten()},
                            index=categorias).reset_index().rename(columns={'index':'Característica'})


df_comparar = pd.melt(df_comparar, id_vars=['Característica'], var_name= "Modelo", value_name="Valor")
print(df_comparar)

  Característica                         Modelo     Valor
0  consumo_medio   308 1.5BlueHDi S&S Style 130 -0.731850
1         precio   308 1.5BlueHDi S&S Style 130 -0.237693
2       potencia   308 1.5BlueHDi S&S Style 130 -0.727229
3    kilometraje   308 1.5BlueHDi S&S Style 130  1.696772
4  consumo_medio  Clase C 220d 4Matic 9G-Tronic -0.839974
5         precio  Clase C 220d 4Matic 9G-Tronic  0.065562
6       potencia  Clase C 220d 4Matic 9G-Tronic -0.835555
7    kilometraje  Clase C 220d 4Matic 9G-Tronic  1.609967


In [172]:
img_a_base64 = encode_image(coche_a["ruta_imagen"])
img_b_base64 = encode_image(coche_b["ruta_imagen"])

fig = px.line_polar(df_comparar, r="Valor", theta="Característica", color="Modelo",
                    line_close=True, title="Comparación de Características de Coches")

fig.update_traces(fill='toself')
fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[-1, max(df_comparar['Valor']) + 0.1])))

fig.add_layout_image(
    dict(
        source= img_a_base64,
        x=-0.3, y=0.5,
        xref="paper", yref="paper",
        sizex=0.4, sizey=0.6,
        xanchor="right", yanchor="middle",
        layer="below"
    )
)

fig.add_layout_image(
    dict(
        source=img_b_base64,
        x=1.25, y=0.5,
        xref="paper", yref="paper",
        sizex=0.6, sizey=0.8,
        xanchor="left", yanchor="middle",
        layer="below"
    )
)

fig.update_layout(
    annotations=[
        dict(
            x=-0.9, y=0.7,  
            xref="paper", yref="paper",
            text=coche_a['modelo_titulo'],
            showarrow=False,
            font=dict(size=14, color="black")
        ),
        dict(
            x=1.9, y=0.7,  
            xref="paper", yref="paper",
            text=coche_b['modelo_titulo'],
            showarrow=False,
            font=dict(size=14, color="black")
        )
    ],
    margin=dict(l=300, r=300), 
    width=900, height=600 
)

fig.show()
